<a href="https://colab.research.google.com/github/cool21th/nmt/blob/master/nmt_with_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

this is


In [0]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time

print(tf.__version__)

1.13.1


In [0]:
path_to_zip = tf.keras.utils.get_file(
'spa-eng.zip', origin='http://download.tensorflow.org/data/spa-eng.zip', 
    extract=True)

path_to_file = os.path.dirname(path_to_zip) + "/spa-eng/spa.txt"

In [0]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
                if unicodedata.category(c) !='Mn')


In [0]:
def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())
  
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.rstrip().strip()
  
  w = '<start> ' + w + ' <end>'
  return w

In [0]:
def create_dataset(path, num_examples):
  lines = open(path, encoding='UTF-8').read().strip().split('\n')
  
  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')] for l in lines[:num_examples]]
 
  return word_pairs

In [0]:
class LanguageIndex():
  def __init__(self, lang):
    self.lang = lang
    self.word2idx = {}
    self.idx2word = {}
    self.vocab = set()
    
    self.create_index()
    
  def create_index(self):
    for phrase in self.lang:
      self.vocab.update(phrase.split(' '))
      
    self.vocab = sorted(self.vocab)
    
    self.word2idx['<pad>'] = 0
    for index, word in enumerate(self.vocab):
      self.word2idx[word] = index +1
    
    for word, index in self.word2idx.items():
      self.idx2word[index] = word
      

In [0]:
def max_length(tensor):
  return max(len(t) for t in tensor)

def load_dataset(path, num_examples):
  pairs = create_dataset(path, num_examples)
  
  inp_lang = LanguageIndex(sp for en, sp in pairs)
  targ_lang = LanguageIndex(en for en, sp in pairs)
  
  input_tensor = [[inp_lang.word2idx[s] for s in sp.split(' ')] for en, sp in pairs]
  
  target_tensor = [[targ_lang.word2idx[s] for s in en.split(' ')] for en, sp in pairs]
  
  max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
  
  input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor,
                                                              maxlen=max_length_inp,
                                                              padding='post')
  
  target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor,
                                                               maxlen=max_length_tar,
                                                               padding='post'
                                                               )
  return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar

In [0]:
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(path_to_file, num_examples)

In [0]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(24000, 24000, 6000, 6000)

In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

The input is put through an encoder model which gives us the encoder output of shape(batch_size, max_length, hidden_size) and the encoder hidden state of shape(batch_size, hidden_size).

Here are the equation that are implemented

Here, the function score is used to compared the target hidden state $$h_t$$ with each of the source  hidden sates $$\overline{h}_s$$, and the result is normalized to produced attention weights ( a distribution  over source positions).
There are various choices of the scoring function; popular scoring functions include the multiplicati ve and additive forms given

In [0]:
def gru(units):
  if tf.test.is_gpu_available():
    return tf.keras.layers.CuDNNGRU(units,
                                   return_sequences=True,
                                   return_state = True,
                                   recurrent_initializer='glorot_uniform')
  else:
    return tf.keras.layers.GRU(units,
                              return_sequences=True,
                              return_state=True,
                              recurrent_activation='sigmoid',
                              recurrent_initializer='glorot_uniform')

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = gru(self.enc_units)
    
  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state
  
  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = gru(self.dec_units)
    self.fc = tf.keras.layers.Dense(vocab_size)
    
    self.W1 = tf.keras.layers.Dense(self.dec_units)
    self.W2 = tf.keras.layers.Dense(self.dec_units)
    self.V = tf.keras.layers.Dense(1)
    
  def call(self, x, hidden , enc_output):
    hidden_with_time_axis = tf.expand_dims(hidden, 1)
    
    score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
    
    attention_weights = tf.nn.softmax(score, axis=1)
    
    context_vector = attention_weights * enc_output
    context_vector = tf.reduce_sum(context_vector, axis=1)
    
    x = self.embedding(x)
    
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
    
    output, state = self.gru(x)
    
    output = tf.reshape(output, (-1, output.shape[2]))
    
    x = self.fc(output)
    
    return x, state, attention_weights
  
  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.dec_units))

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [0]:
optimizer = tf.train.AdamOptimizer()

def loss_function(real, pred):
  mask = 1 - np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * maks
  return tf.reduce_mean(loss_)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint =  tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [0]:
EPOCHS =10

for epoch in range(EPOCHS):
  start = time.time()
  
  hidden = encoder.initialize_hidden_state()
  total_loss = 0
  
  for (batch, (inp, targ)) in enumerate(dataset):
    loss = 0
    
    with tf.GradientTape() as tape:
      enc_output, enc_hidden = encoder(inp, hidden)
      
      dec_hidden = enc_hidden
      
      dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE,1)
      
      for t in range(1, targ.shape[1]):
        predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
        
        loss += loss_function(targ[:, t], predictions)
        
        dec_input = tf.expand_dims(targ[:, t], 1)
        
    batch_loss = (loss / int(targ.shape[1]))
    
    total_loss +=  batch_loss
    
    variables = encoder.variables + decoder.variables
    
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, varialbes))
    
    
    if batch % 100 ==0 :
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
  if (epoch + 1) % 2 == 0:
    checkpon.save(file_prefix = checkpoin_prefix)
    
  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

KeyError: ignored